# <font color="orange"> House Prices - Advanced Regression Techniques </font>

#### Predict sales prices and practice feature engineering, RFs, and gradient boosting

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [26]:
DataFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Datasets/house-prices-advanced-regression-techniques/"
OutputFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Output"

In [27]:
hdf = pd.read_csv(DataFolder+"train.csv")
hdf.reset_index(drop=True, inplace=True)

In [28]:
hdf

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [29]:
hdf_y = hdf["SalePrice"]

In [30]:
hdf.drop(columns=["SalePrice","Id"],inplace=True)

In [35]:
hdf["MSSubClass"] = hdf["MSSubClass"].astype("object")
hdf.dtypes

MSSubClass        object
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

In [36]:
hdf[hdf.columns[hdf.isnull().any()]].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [34]:
hdf[hdf.columns[hdf.dtypes != "Object"]]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


### Missing value treatment

In [38]:
# columns treatment
for col in hdf.columns:
    nullVals = round(hdf[col].isnull().sum()/len(hdf)*100,2)
    if nullVals > 50:
        hdf.drop(columns=col, axis=1, inplace=True)

In [41]:
# row treatment
for i in range(len(hdf)):
    nullVals = hdf.loc[i].isnull().sum()/len(hdf.columns)*100
    if nullVals > 50:
        hdf.drop(i, axis=0, inplace=True)

In [42]:
hdf.shape

(1460, 75)

### Separating Numerical and Categorical Data

In [44]:
hdf_num = hdf[hdf.columns[hdf.dtypes != "object"]]
hdf_cat = hdf[hdf.columns[hdf.dtypes == "object"]]

In [45]:
hdf_num[hdf_num.columns[hdf_num.isnull().any()]].isnull().sum()

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [46]:
hdf_cat[hdf_cat.columns[hdf_cat.isnull().any()]].isnull().sum()

MasVnrType        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

### Imputation

In [53]:
from sklearn.impute import KNNImputer

In [54]:
imputer = KNNImputer(n_neighbors=3)
hdf_num_imputed = imputer.fit_transform(hdf_num)

In [55]:
hdf_num_imputed = pd.DataFrame(hdf_num_imputed,columns=hdf_num.columns)

In [56]:
from sklearn.impute import SimpleImputer

In [57]:
simpleImpute = SimpleImputer(strategy="most_frequent")

In [59]:
hdf_cat_imputed = pd.DataFrame(simpleImpute.fit_transform(hdf_cat),columns=hdf_cat.columns)

In [65]:
print(hdf_num_imputed.isnull().sum().sum())
print(hdf_cat_imputed.isnull().sum().sum())

0
0


### Outlier Treatment

In [47]:
from scipy.stats import zscore

In [51]:
import numpy as np
zscores = zscore(hdf_num)

In [52]:
zscores < 3

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,False,True,True,True,True,True,False,True,True,True,...,True,True,True,False,True,True,True,True,True,True
4,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1456,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1457,False,True,True,False,True,True,False,True,True,True,...,True,True,True,True,True,True,True,False,True,True
1458,False,True,True,True,True,True,False,True,False,True,...,True,True,True,True,True,True,True,True,True,True
